<a href="https://colab.research.google.com/github/jonanew/noise_detection/blob/main/Final_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install rdflib for KG
!pip install rdflib

# Install noise package for noise generation
!pip install noise

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp311-cp311-linux_x86_64.whl size=56278 sha256=9c41a14f1f3428a8b523e5bc4605803eb92dd21c2919b0ad3a39d22a8400c881
  Stored in directory: /root/.cache/pip/wheels/39/25/2e/af6d1bcc91a8f99af0f651f8718b9ab999720a21c6d4149091
Successfully built noise


In [2]:
# Connect to my drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import enum
import pandas as pd
import numpy as np
import networkx as nx
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD
import sys

def networkx(rdf_graph):
    """
    Convert an RDFLib graph to a NetworkX graph.
    - Nodes: All individuals and ontology classes.
    - Edges: Object properties create directed edges between nodes.
    - Attributes: Data properties are stored as node attributes.
    """
    if isinstance(rdf_graph, Graph) or isinstance(rdf_graph, str):
        if isinstance(rdf_graph, Graph):
            rdf_graph = rdf_graph
        elif isinstance(rdf_graph, str):
            if rdf_graph.endswith('.ttl'):
                try:
                    rdf_graph = Graph().parse(rdf_graph, format='turtle')
                except Exception as e:
                    print(f"Error parsing Turtle file: {e}")
                    return None
            else:
                print("Unexpected input format. Provide a Turtle (.ttl) file path.")
    else:
        print("Unexpected input format. Provide an RDFLib Graph or a .ttl file path.")
        return None

    # Initialize the directed graph
    nx_graph = nx.DiGraph()

    # Process the RDF graph
    for s, p, o in rdf_graph:
        s, p = s.toPython(), p.toPython()
        # Ensure subject is a node
        if s not in nx_graph:
            nx_graph.add_node(s)

        # Check if the object is a literal (data property) or an entity (object property)
        if isinstance(o, Literal):  # Data property → store as a node attribute
            nx_graph.nodes[s][p] = o.toPython()
        else:  # Object property → add an edge
            o = o.toPython()
            # in case o is not a hashable type, parse it to string
            if isinstance(o, enum.Enum):
                o = str(o)
            if p == RDF.type.toPython():
                nx_graph.nodes[s][RDF.type.toPython()] = o
            else:
                nx_graph.add_edge(s, o, predicate=p)

    return nx_graph

In [4]:
import enum

import networkx as nx
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD
import sys

def networkx(rdf_graph):
    """
    Convert an RDFLib graph to a NetworkX graph.
    - Nodes: All individuals and ontology classes.
    - Edges: Object properties create directed edges between nodes.
    - Attributes: Data properties are stored as node attributes.
    """
    if isinstance(rdf_graph, Graph) or isinstance(rdf_graph, str):
        if isinstance(rdf_graph, Graph):
            rdf_graph = rdf_graph
        elif isinstance(rdf_graph, str):
            if rdf_graph.endswith('.ttl'):
                try:
                    rdf_graph = Graph().parse(rdf_graph, format='turtle')
                except Exception as e:
                    print(f"Error parsing Turtle file: {e}")
                    return None
            else:
                print("Unexpected input format. Provide a Turtle (.ttl) file path.")
    else:
        print("Unexpected input format. Provide an RDFLib Graph or a .ttl file path.")
        return None

    # Initialize the directed graph
    nx_graph = nx.DiGraph()

    # Process the RDF graph
    for s, p, o in rdf_graph:
        s, p = s.toPython(), p.toPython()
        # Ensure subject is a node
        if s not in nx_graph:
            nx_graph.add_node(s)

        # Check if the object is a literal (data property) or an entity (object property)
        if isinstance(o, Literal):  # Data property → store as a node attribute
            nx_graph.nodes[s][p] = o.toPython()
        else:  # Object property → add an edge
            o = o.toPython()
            # in case o is not a hashable type, parse it to string
            if isinstance(o, enum.Enum):
                o = str(o)
            if p == RDF.type.toPython():
                nx_graph.nodes[s][RDF.type.toPython()] = o
            else:
                nx_graph.add_edge(s, o, predicate=p)

    return nx_graph

In [5]:
import re
from rdflib import RDF

def shorten_uri(uri, ontology_base):
    """
    Extract the local name from a full URI.
    Removes known ontology base and keeps suffix like Pipe_9 or diameter.
    """
    if uri.startswith(ontology_base):
        return uri.replace(ontology_base, "")
    elif "#" in uri:
        return uri.split("#")[-1]
    elif "/" in uri:
        return uri.split("/")[-1]
    else:
        return uri

def extract_features_from_nx_graph(nx_graph, ontology_base, edge_predicates, node_type_filter, attribute_keys):

    feature_dict = {}
    rdf_type_key = shorten_uri(str(RDF.type), ontology_base)

    for node, data in nx_graph.nodes(data=True):
        #print(f"Processing node: {node}, data: {data}")

        # Shorten RDF.type value and all keys
        cleaned_data = {
            shorten_uri(k, ontology_base): v if not isinstance(v, str) else shorten_uri(v, ontology_base)
            for k, v in data.items()
        }

        #print(f"Cleaned data: {cleaned_data}")

        # Filter by type if specified
        node_type = cleaned_data.get(rdf_type_key)
        if node_type_filter and node_type != node_type_filter:
            continue

        # Shorten node URI
        node_id = shorten_uri(node, ontology_base)

        # Select attributes
        if attribute_keys:
            features = {k: cleaned_data[k] for k in attribute_keys if k in cleaned_data}
        else:
            # Remove rdf:type from features
            features = {k: v for k, v in cleaned_data.items() if k != rdf_type_key}

        # Now add edges as attributes if requested
        if edge_predicates:
            for _, target, edge_data in nx_graph.out_edges(node, data=True):
                pred = shorten_uri(edge_data.get("predicate", ""), ontology_base)
                if pred in edge_predicates:
                    features[pred] = shorten_uri(target, ontology_base)

        feature_dict[node_id] = features

    return feature_dict


In [6]:


# Initialize an empty NetworkX graph to store the merged knowledge graph
kg_graph = nx.DiGraph()  # Assuming a directed graph; use nx.Graph() if undirected

# Define the base path pattern and iterate over files 1 to 10
base_path = "/content/drive/My Drive/Master Thesis Semantics/LeakDB/Knowledge Graphs/knowledge_graph{}-LeakDB.ttl"
for i in range(1, 11):  # Iterate from 1 to 10
    kg_path = base_path.format(i)

    # Convert the KG file to a NetworkX graph
    try:
        single_kg_graph = networkx(kg_path)  # Assuming networkx() converts .ttl to NetworkX graph
        if single_kg_graph is not None:
            # Merge the single graph into the main graph
            kg_graph = nx.compose(kg_graph, single_kg_graph)
        else:
            print(f"Failed to load knowledge graph from {kg_path}. Skipping.")
    except Exception as e:
        print(f"Error loading {kg_path}: {str(e)}. Skipping.")

# Check if the merged graph has nodes/edges before proceeding
if kg_graph.number_of_nodes() > 0:
    pipe_features = extract_features_from_nx_graph(
        kg_graph,  # Pass the merged NetworkX graph
        "https://raw.githubusercontent.com/DiTEC-project/wdn-knowledge-graph/refs/heads/main/wdn_ontology.ttl",
        node_type_filter="Pipe",
        attribute_keys=["diameter", "length", "roughness", "status"],
        edge_predicates=None
    )
else:
    print("Failed to load any knowledge graphs. Cannot extract features.")
    pipe_features = {}  # Initialize pipe_features as an empty dictionary if loading failed

In [7]:
pipe_features

{'Pipe_27': {'diameter': 0.3393533,
  'length': 256.3145,
  'roughness': 120.9279,
  'status': 'Open'},
 'Pipe_31': {'diameter': 0.2844319,
  'length': 1777.14,
  'roughness': 122.255,
  'status': 'Open'},
 'Pipe_9': {'diameter': 1.055765,
  'length': 766.6966,
  'roughness': 134.3591,
  'status': 'Open'},
 'Pipe_10': {'diameter': 0.8135806,
  'length': 943.0429,
  'roughness': 122.1229,
  'status': 'Open'},
 'Pipe_25': {'diameter': 0.6257758,
  'length': 1249.409,
  'roughness': 126.3149,
  'status': 'Open'},
 'Pipe_7': {'diameter': 0.8599014,
  'length': 874.319,
  'roughness': 148.771,
  'status': 'Open'},
 'Pipe_1': {'diameter': 1.090263,
  'length': 102.9672,
  'roughness': 132.6247,
  'status': 'Open'},
 'Pipe_32': {'diameter': 0.3227161,
  'length': 156.6218,
  'roughness': 131.7815,
  'status': 'Open'},
 'Pipe_21': {'diameter': 0.4833052,
  'length': 1353.826,
  'roughness': 123.2031,
  'status': 'Open'},
 'Pipe_14': {'diameter': 0.3435959,
  'length': 486.0105,
  'roughness': 

In [8]:
pipe_df = pd.DataFrame.from_dict(pipe_features, orient='index').reset_index()
pipe_df.columns = ['pipe_id', 'diameter', 'length', 'roughness', 'status']

In [9]:
pipe_df

,pipe_id,diameter,length,roughness,status
0,Pipe_27,0.339353,256.3145,120.9279,Open
1,Pipe_31,0.284432,1777.1400,122.2550,Open
2,Pipe_9,1.055765,766.6966,134.3591,Open
3,Pipe_10,0.813581,943.0429,122.1229,Open
4,Pipe_25,0.625776,1249.4090,126.3149,Open
5,Pipe_7,0.859901,874.3190,148.7710,Open
6,Pipe_1,1.090263,102.9672,132.6247,Open
7,Pipe_32,0.322716,156.6218,131.7815,Open
8,Pipe_21,0.483305,1353.8260,123.2031,Open
9,Pipe_14,0.343596,486.0105,121.7812,Open


In [10]:
import pandas as pd

# Initialize an empty list to store individual DataFrames
dataframes = []

# Define the base path pattern and iterate over files 1 to 10
base_path = "/content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_{}_LeakDB.csv"
# Load and combine CSV files
for i in range(1, 11):
    csv_path = base_path.format(i)
    try:
        df = pd.read_csv(csv_path)
        if not df.empty:
            dataframes.append(df)
            print(f"Loaded {csv_path} with {len(df)} rows.")
        else:
            print(f"Warning: {csv_path} is empty. Skipping.")
    except FileNotFoundError:
        print(f"Error: {csv_path} not found. Skipping.")
    except Exception as e:
        print(f"Error loading {csv_path}: {str(e)}. Skipping.")

# Combine CSV files into a single DataFrame
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"Combined CSV data has {len(combined_df)} rows.")
else:
    print("No CSV files loaded. Exiting.")
    exit(1)

# Filter for flow measurements and create pipe_id
combined_df = combined_df[combined_df['sensor_type'] == 'flow'].copy()
combined_df['pipe_id'] = combined_df['sensor_id'].str.replace('Link_', 'Pipe_').str.replace('_flow', '')

Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_1_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_2_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_3_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_4_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_5_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_6_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_7_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CSV Files/measurements_8_LeakDB.csv with 1716960 rows.
Loaded /content/drive/My Drive/Master Thesis Semantics/LeakDB/CS

In [11]:
combined_df

,Unnamed: 0,Timestamp,measurement,sensor_id,sensor_type,unique_id,pipe_id
1121280,1121280,2017-01-01 00:00:00,3405.6,Link_1_flow,flow,Link_1_flow_2017-01-01 00:00:00,Pipe_1
1121281,1121281,2017-01-01 00:30:00,2970.0,Link_1_flow,flow,Link_1_flow_2017-01-01 00:30:00,Pipe_1
1121282,1121282,2017-01-01 01:00:00,2656.8,Link_1_flow,flow,Link_1_flow_2017-01-01 01:00:00,Pipe_1
1121283,1121283,2017-01-01 01:30:00,2401.2,Link_1_flow,flow,Link_1_flow_2017-01-01 01:30:00,Pipe_1
1121284,1121284,2017-01-01 02:00:00,2199.6,Link_1_flow,flow,Link_1_flow_2017-01-01 02:00:00,Pipe_1
...,...,...,...,...,...,...,...
17169595,1716955,2017-12-31 21:30:00,925.2,Link_9_flow,flow,Link_9_flow_2017-12-31 21:30:00,Pipe_9
17169596,1716956,2017-12-31 22:00:00,864.0,Link_9_flow,flow,Link_9_flow_2017-12-31 22:00:00,Pipe_9
17169597,1716957,2017-12-31 22:30:00,806.4,Link_9_flow,flow,Link_9_flow_2017-12-31 22:30:00,Pipe_9
17169598,1716958,2017-12-31 23:00:00,781.2,Link_9_flow,flow,Link_9_flow_2017-12-31 23:00:00,Pipe_9


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from itertools import combinations
import warnings

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined from previous steps
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.6f}, Std = {global_flow_std:.6f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

# Compute global pipe std for minimum threshold
global_pipe_std = pipe_stats_df['actual_std'].mean()

# Apply a minimum std to avoid overly narrow ranges (10% of global pipe std)
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges using pipe-level stats with fixed k
k = 3  # Fixed number of standard deviations for plausible range
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
print("Per-pipe plausible ranges (fixed k):\n", pipe_stats_df[['pipe_id', 'actual_mean', 'actual_std', 'min_plausible', 'max_plausible']])

# Merge the plausible ranges into the main DataFrame
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Subsample the data (10%)
subsample_fraction = 0.1
subsample_size = int(len(merged_df) * subsample_fraction)
merged_df = merged_df.sample(n=subsample_size, random_state=42)
print(f"Subsampled data size: {len(merged_df)}")

# Define all possible feature combinations
base_feature = ['measurement']
semantic_features = ['diameter', 'length', 'roughness']
all_combinations = []
for r in range(1, len(semantic_features) + 1):
    for combo in combinations(semantic_features, r):
        all_combinations.append(base_feature + list(combo))
all_combinations = [base_feature] + all_combinations
print(f"Total feature combinations to test: {len(all_combinations)}")

# Function to introduce General Normalised Noise (GNN)
def introduce_gnn_noise(df, beta=1.5, noise_fraction=0.2):
    noisy_df = df.copy()
    mask = np.random.rand(len(noisy_df)) < noise_fraction
    noise = np.random.normal(0, beta * global_flow_std, size=len(noisy_df))
    noisy_df['measurement'] = noisy_df['measurement'] + noise * mask
    noisy_df['is_true_noise'] = mask.astype(int)
    return noisy_df

# Function to evaluate a feature combination with cross-validation
def evaluate_feature_combination(features, df, n_clusters=7, threshold=2, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
        train_df = df.iloc[train_idx].copy()
        test_df = df.iloc[test_idx].copy()

        # Scale features
        X_cluster_train = train_df[features]
        scaler = RobustScaler()
        X_scaled_train = scaler.fit_transform(X_cluster_train)

        # Apply K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        train_cluster_labels = kmeans.fit_predict(X_scaled_train)

        X_cluster_test = test_df[features]
        X_scaled_test = scaler.transform(X_cluster_test)
        test_cluster_labels = kmeans.predict(X_scaled_test)

        train_df['cluster'] = train_cluster_labels
        test_df['cluster'] = test_cluster_labels

        # Compute cluster statistics
        cluster_stats = train_df.groupby('cluster').agg({
            'measurement': ['mean', 'std', 'count']
        }).reset_index()
        cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count']

        # Map cluster stats to test set
        cluster_means = cluster_stats.set_index('cluster')['cluster_mu'].to_dict()
        cluster_stds = cluster_stats.set_index('cluster')['cluster_sigma'].to_dict()
        test_df['cluster_mu'] = test_df['cluster'].map(cluster_means)
        test_df['cluster_sigma'] = test_df['cluster'].map(cluster_stds).fillna(global_pipe_std)

        # Introduce noise to test set
        noisy_test_df = introduce_gnn_noise(test_df, beta=1.5, noise_fraction=0.2)

        # Compute Z-scores and detect anomalies
        noisy_test_df['Z_score'] = (noisy_test_df['measurement'] - noisy_test_df['cluster_mu']) / noisy_test_df['cluster_sigma'].replace(0, np.nan)
        noisy_test_df['traditional_Z_score'] = (noisy_test_df['measurement'] - global_flow_mean) / global_flow_std
        noisy_test_df['outside_plausible'] = (noisy_test_df['measurement'] < noisy_test_df['min_plausible']) | (noisy_test_df['measurement'] > noisy_test_df['max_plausible'])
        noisy_test_df['is_anomaly'] = np.abs(noisy_test_df['Z_score']) > threshold
        noisy_test_df['is_noise'] = noisy_test_df['is_anomaly'] & noisy_test_df['outside_plausible']

        # Evaluate performance
        y_true = noisy_test_df['is_true_noise']
        y_pred = noisy_test_df['is_noise'].astype(int)

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        total_noise_points = y_true.sum()
        detected_noise = y_pred[y_true == 1].sum()
        noise_detected_percentage = (detected_noise / total_noise_points * 100) if total_noise_points > 0 else 0

        fold_results.append({
            'fold': fold + 1,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'noise_detected_percentage': noise_detected_percentage
        })

    return fold_results

# Evaluate all feature combinations
results_list = []
for idx, feature_combo in enumerate(all_combinations):
    print(f"Testing feature combination: {feature_combo}")
    fold_results = evaluate_feature_combination(feature_combo, merged_df, n_clusters=7, threshold=2)

    # Aggregate fold results
    avg_f1 = np.mean([result['f1_score'] for result in fold_results])
    last_fold = fold_results[-1]  # Use last fold for precision, recall, noise detected (as in DBSCAN setup)

    results_list.append({
        'Features': ', '.join(feature_combo),
        'Avg F1 Score': avg_f1,
        'Precision': last_fold['precision'],
        'Recall': last_fold['recall'],
        'Noise Detected (%)': last_fold['noise_detected_percentage']
    })

    # Print fold-level results
    for result in fold_results:
        print(f"Fold {result['fold']} Results - Features: {feature_combo}")
        print(f"  Precision: {result['precision']:.6f}")
        print(f"  Recall: {result['recall']:.6f}")
        print(f"  F1 Score: {result['f1_score']:.6f}")
        print(f"  Noise Detected (%): {result['noise_detected_percentage']:.6f}")

# Create results DataFrame and sort by Avg F1 Score
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by='Avg F1 Score', ascending=False).reset_index(drop=True)
print("\nFeature Selection Results (sorted by Avg F1 Score):")
print(results_df)

# Save results
try:
    results_df.to_csv('/content/drive/My Drive/feature_selection_results_kmeans_leakdb.csv', index=False)
    print("Feature selection results saved to '/content/drive/My Drive/feature_selection_results_kmeans_leakdb.csv'.")
except Exception as e:
    print(f"Error saving file: {e}")

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.876609, Std = 1495.636123
Per-pipe plausible ranges (fixed k):
     pipe_id  actual_mean   actual_std  min_plausible  max_plausible
0    Pipe_1  5627.037288  1718.679788     470.997924   10783.076651
1   Pipe_10   550.314267   195.561336     -36.369742    1136.998276
2   Pipe_11   412.248822   162.662078     -75.737413     900.235057
3   Pipe_12   253.255767    83.374362       3.132682     503.378853
4   Pipe_13   266.276404    99.849458     -33.271971     565.824779
5   Pipe_14    88.363952    60.337333     -92.648046     269.375950
6   Pipe_15    13.391692    56.662783    -156.596656     183.380040
7   Pipe_16   156.229705    78.588859     -79.536872     391.996283
8   Pipe_17  -405.530240   134.493077    -809.009471      -2.051009
9   Pipe_18  -777.124479   242.406106   -1504.342797     -49.906162
10  Pipe_19  -794.791479   247.168817   -1536.297930     -53.285029
11   Pipe_2  5386.296760  1649.485821     437.839298   103

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
import itertools

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined from previous steps
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

# Compute global pipe std for minimum threshold
global_pipe_std = pipe_stats_df['actual_std'].mean()

# Apply a minimum std to avoid overly narrow ranges (10% of global pipe std)
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges using pipe-level stats with fixed k
k = 3  # Fixed number of standard deviations for plausible range
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
print("Per-pipe plausible ranges (fixed k):\n", pipe_stats_df[['pipe_id', 'actual_mean', 'actual_std', 'min_plausible', 'max_plausible']])

# Merge the plausible ranges into the main DataFrame
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Use full dataset (100% with no subsampling)
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

# Define all possible feature combinations
all_features = ['measurement', 'diameter', 'length', 'roughness']
feature_combinations = []
for r in range(1, len(all_features) + 1):
    feature_combinations.extend(list(itertools.combinations(all_features, r)))

results_list = []

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for features in feature_combinations:
    features = list(features)
    print(f"Testing feature combination: {features}")

    fold_f1_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        fold_train_df = train_df.iloc[train_idx]
        fold_val_df = train_df.iloc[val_idx]

        # Scale features
        X_cluster_train = fold_train_df[features]
        scaler = RobustScaler()
        X_scaled_train = scaler.fit_transform(X_cluster_train)

        # Determine optimal number of clusters using elbow method (simplified for each fold)
        wcss = []
        for i in range(2, 12):
            kmeans = KMeans(n_clusters=i, random_state=42)
            kmeans.fit(X_scaled_train)
            wcss.append(kmeans.inertia_)
        optimal_clusters = 7  # Fixed based on previous analysis, adjust if needed

        kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
        train_cluster_labels = kmeans.fit_predict(X_scaled_train)

        X_cluster_val = fold_val_df[features]
        X_scaled_val = scaler.transform(X_cluster_val)
        val_cluster_labels = kmeans.predict(X_scaled_val)

        fold_train_df['cluster'] = train_cluster_labels
        fold_val_df['cluster'] = val_cluster_labels

        # Compute cluster statistics on training fold
        cluster_stats = fold_train_df.groupby('cluster').agg({
            'measurement': ['mean', 'std', 'count'],
            'pipe_id': lambda x: list(x.unique())
        }).reset_index()
        cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
        cluster_stats['cluster_mu'] = cluster_stats['cluster_mu'].round(2)
        cluster_stats['cluster_sigma'] = cluster_stats['cluster_sigma'].round(2)

        cluster_means = fold_train_df.groupby('cluster')['measurement'].mean().to_dict()
        cluster_stds = fold_train_df.groupby('cluster')['measurement'].std().to_dict()
        fold_val_df['cluster_mu'] = fold_val_df['cluster'].map(cluster_means)
        fold_val_df['cluster_sigma'] = fold_val_df['cluster'].map(cluster_stds).fillna(global_pipe_std)

        fold_val_df['Z_score'] = (fold_val_df['measurement'] - fold_val_df['cluster_mu']) / fold_val_df['cluster_sigma'].replace(0, np.nan)
        fold_val_df['traditional_Z_score'] = (fold_val_df['measurement'] - global_flow_mean) / global_flow_std

        fold_val_df['outside_plausible'] = (fold_val_df['measurement'] < fold_val_df['min_plausible']) | (fold_val_df['measurement'] > fold_val_df['max_plausible'])
        fold_val_df['is_anomaly'] = np.abs(fold_val_df['Z_score']) > 3
        fold_val_df['is_anomaly_traditional'] = np.abs(fold_val_df['traditional_Z_score']) > 3
        fold_val_df['is_noise'] = fold_val_df['is_anomaly'] & fold_val_df['outside_plausible']

        # Introduce noise for this fold
        noisy_val_df = introduce_gaussian_noise(fold_val_df, noise_level=global_flow_std, noise_fraction=0.20)

        noisy_val_df['Z_score'] = (noisy_val_df['measurement'] - noisy_val_df['cluster_mu']) / noisy_val_df['cluster_sigma'].replace(0, np.nan)
        noisy_val_df['traditional_Z_score'] = (noisy_val_df['measurement'] - global_flow_mean) / global_flow_std
        noisy_val_df['outside_plausible'] = (noisy_val_df['measurement'] < noisy_val_df['min_plausible']) | (noisy_val_df['measurement'] > noisy_val_df['max_plausible'])
        noisy_val_df['is_anomaly'] = np.abs(noisy_val_df['Z_score']) > 3
        noisy_val_df['is_anomaly_traditional'] = np.abs(noisy_val_df['traditional_Z_score']) > 3
        noisy_val_df['is_noise'] = noisy_val_df['is_anomaly'] & noisy_val_df['outside_plausible']

        y_true = noisy_val_df['is_true_noise']
        y_pred_hybrid = noisy_val_df['is_noise'].astype(int)

        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)
        fold_f1_scores.append(f1_hybrid)

    avg_f1_score = np.mean(fold_f1_scores)
    std_f1_score = np.std(fold_f1_scores)
    results_list.append({
        'Feature Combination': features,
        'Avg F1 Score': avg_f1_score,
        'Std F1 Score': std_f1_score
    })

# Sort results by Avg F1 Score
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by='Avg F1 Score', ascending=False)
print("\nFeature Selection Results:")
print(results_df)

# Save results
try:
    results_df.to_csv('/content/drive/My Drive/feature_selection_results_kmeans_flow.csv', index=False)
    print("Results saved to '/content/drive/My Drive/'.")
except Exception as e:
    print(f"Error saving files: {e}")

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Per-pipe plausible ranges (fixed k):
     pipe_id  actual_mean   actual_std  min_plausible  max_plausible
0    Pipe_1  5627.037288  1718.679788     470.997924   10783.076651
1   Pipe_10   550.314267   195.561336     -36.369742    1136.998276
2   Pipe_11   412.248822   162.662078     -75.737413     900.235057
3   Pipe_12   253.255767    83.374362       3.132682     503.378853
4   Pipe_13   266.276404    99.849458     -33.271971     565.824779
5   Pipe_14    88.363952    60.337333     -92.648046     269.375950
6   Pipe_15    13.391692    56.662783    -156.596656     183.380040
7   Pipe_16   156.229705    78.588859     -79.536872     391.996283
8   Pipe_17  -405.530240   134.493077    -809.009471      -2.051009
9   Pipe_18  -777.124479   242.406106   -1504.342797     -49.906162
10  Pipe_19  -794.791479   247.168817   -1536.297930     -53.285029
11   Pipe_2  5386.296760  1649.485821     437.839298   10334.75422

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import f1_score
import warnings
import itertools

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined from previous steps
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

# Compute global pipe std for minimum threshold
global_pipe_std = pipe_stats_df['actual_std'].mean()

# Apply a minimum std to avoid overly narrow ranges (10% of global pipe std)
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges using pipe-level stats with fixed k
k = 3  # Fixed number of standard deviations for plausible range
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
print("Per-pipe plausible ranges (fixed k):\n", pipe_stats_df[['pipe_id', 'actual_mean', 'actual_std', 'min_plausible', 'max_plausible']])

# Merge the plausible ranges into the main DataFrame
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Subsample the data to 10% as requested
subsample_fraction = 0.05  # Use 10% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Split into training and test sets
train_df, test_df = train_test_split(subsampled_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

# Define all possible feature combinations
all_features = ['measurement', 'diameter', 'length', 'roughness']
feature_combinations = []
for r in range(1, len(all_features) + 1):
    feature_combinations.extend(list(itertools.combinations(all_features, r)))

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

def introduce_gaussian_noise(df, noise_level=1495.64, noise_fraction=0.05):
    noisy_df = df.copy()
    mask = np.random.rand(len(noisy_df)) < noise_fraction
    noise = np.random.normal(0, noise_level, size=len(noisy_df))
    noisy_df['measurement'] = noisy_df['measurement'] + noise * mask
    noisy_df['is_true_noise'] = mask.astype(int)
    return noisy_df

for features in feature_combinations:
    features = list(features)
    print(f"Testing feature combination: {features}")

    fold_f1_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        fold_train_df = train_df.iloc[train_idx]
        fold_val_df = train_df.iloc[val_idx]

        # Scale features
        X_cluster_train = fold_train_df[features]
        scaler = RobustScaler()
        X_scaled_train = scaler.fit_transform(X_cluster_train)

        # Generate k-distance plot to determine eps (on a smaller sample for efficiency)
        min_samples = 50
        neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
        distances, indices = neighbors.kneighbors(X_scaled_train)
        k_distances = np.sort(distances[:, min_samples - 1])
        suggested_eps = np.percentile(k_distances, 95)

        # Ensure suggested_eps is positive
        suggested_eps = max(suggested_eps, 0.01)  # Minimum eps value

        # Parameter tuning using silhouette score
        eps_values = np.arange(max(0.005, suggested_eps - 0.005), suggested_eps + 0.005, 0.002)
        min_samples_values = [20, 30, 50, 60, 70]
        best_score = -1
        best_params = {'eps': suggested_eps, 'min_samples': 50}  # Default to suggested_eps and min_samples

        for eps in eps_values:
            for min_samples in min_samples_values:
                dbscan = DBSCAN(eps=eps, min_samples=min_samples)
                labels = dbscan.fit_predict(X_scaled_train)
                if len(np.unique(labels)) > 1 and -1 in labels:  # Ensure multiple clusters and noise points
                    score = silhouette_score(X_scaled_train, labels, sample_size=5000)
                    if score > best_score:
                        best_score = score
                        best_params = {'eps': eps, 'min_samples': min_samples}

        # If no valid parameters found, log and use default
        if best_params['eps'] == suggested_eps and best_params['min_samples'] == 50:
            print(f"  Fold {fold}: No valid parameters found for {features}. Using default eps={suggested_eps:.3f}, min_samples=50")

        # Use best parameters for clustering
        dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
        train_cluster_labels = dbscan.fit_predict(X_scaled_train)

        X_cluster_val = fold_val_df[features]
        X_scaled_val = scaler.transform(X_cluster_val)
        val_cluster_labels = dbscan.fit_predict(X_scaled_val)

        fold_train_df['cluster'] = train_cluster_labels
        fold_val_df['cluster'] = val_cluster_labels

        # Compute cluster statistics (non-noise clusters)
        cluster_stats = fold_train_df[fold_train_df['cluster'] != -1].groupby('cluster').agg({
            'measurement': ['mean', 'std', 'count'],
            'pipe_id': lambda x: list(x.unique())
        }).reset_index()
        cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
        cluster_stats['cluster_mu'] = cluster_stats['cluster_mu'].round(2)
        cluster_stats['cluster_sigma'] = cluster_stats['cluster_sigma'].round(2)

        # Merge small clusters
        min_cluster_size = 100
        small_clusters = cluster_stats[cluster_stats['point_count'] < min_cluster_size]['cluster'].tolist()
        fold_val_df.loc[fold_val_df['cluster'].isin(small_clusters), 'cluster'] = -1

        # Recompute cluster statistics
        cluster_stats = fold_train_df[fold_train_df['cluster'] != -1].groupby('cluster').agg({
            'measurement': ['mean', 'std', 'count'],
            'pipe_id': lambda x: list(x.unique())
        }).reset_index()
        cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']

        cluster_means = fold_train_df[fold_train_df['cluster'] != -1].groupby('cluster')['measurement'].mean().to_dict()
        cluster_stds = fold_train_df[fold_train_df['cluster'] != -1].groupby('cluster')['measurement'].std().to_dict()
        min_sigma = global_pipe_std * 0.1
        cluster_stds = {k: max(v, min_sigma) for k, v in cluster_stds.items()}
        fold_val_df['cluster_mu'] = fold_val_df['cluster'].map(cluster_means)
        fold_val_df['cluster_sigma'] = fold_val_df['cluster'].map(cluster_stds).fillna(global_pipe_std)
        fold_val_df.loc[fold_val_df['cluster'] == -1, 'cluster_mu'] = global_flow_mean
        fold_val_df.loc[fold_val_df['cluster'] == -1, 'cluster_sigma'] = global_flow_std

        fold_val_df['Z_score'] = (fold_val_df['measurement'] - fold_val_df['cluster_mu']) / fold_val_df['cluster_sigma'].replace(0, np.nan)
        fold_val_df['outside_plausible'] = (fold_val_df['measurement'] < fold_val_df['min_plausible']) | (fold_val_df['measurement'] > fold_val_df['max_plausible'])
        fold_val_df['is_anomaly'] = np.abs(fold_val_df['Z_score']) > 3
        fold_val_df['is_noise'] = fold_val_df['is_anomaly'] & fold_val_df['outside_plausible']

        # Introduce noise (β=1, 20% noise fraction)
        noisy_val_df = introduce_gaussian_noise(fold_val_df, noise_level=global_flow_std, noise_fraction=0.20)

        noisy_val_df['Z_score'] = (noisy_val_df['measurement'] - noisy_val_df['cluster_mu']) / noisy_val_df['cluster_sigma'].replace(0, np.nan)
        noisy_val_df['outside_plausible'] = (noisy_val_df['measurement'] < noisy_val_df['min_plausible']) | (noisy_val_df['measurement'] > noisy_val_df['max_plausible'])
        noisy_val_df['is_anomaly'] = np.abs(noisy_val_df['Z_score']) > 3
        noisy_val_df['is_noise'] = noisy_val_df['is_anomaly'] & noisy_val_df['outside_plausible']

        y_true = noisy_val_df['is_true_noise']
        y_pred_hybrid = noisy_val_df['is_noise'].astype(int)

        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)
        fold_f1_scores.append(f1_hybrid)

    avg_f1_score = np.mean(fold_f1_scores)
    std_f1_score = np.std(fold_f1_scores)
    results_list.append({
        'Feature Combination': features,
        'Avg F1 Score': avg_f1_score,
        'Std F1 Score': std_f1_score
    })

# Sort results by Avg F1 Score
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by='Avg F1 Score', ascending=False)
print("\nFeature Selection Results:")
print(results_df)

# Save results
try:
    results_df.to_csv('/content/drive/My Drive/feature_selection_results_dbscan_flow.csv', index=False)
    print("Results saved to '/content/drive/My Drive/'.")
except Exception as e:
    print(f"Error saving files: {e}")

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Per-pipe plausible ranges (fixed k):
     pipe_id  actual_mean   actual_std  min_plausible  max_plausible
0    Pipe_1  5627.037288  1718.679788     470.997924   10783.076651
1   Pipe_10   550.314267   195.561336     -36.369742    1136.998276
2   Pipe_11   412.248822   162.662078     -75.737413     900.235057
3   Pipe_12   253.255767    83.374362       3.132682     503.378853
4   Pipe_13   266.276404    99.849458     -33.271971     565.824779
5   Pipe_14    88.363952    60.337333     -92.648046     269.375950
6   Pipe_15    13.391692    56.662783    -156.596656     183.380040
7   Pipe_16   156.229705    78.588859     -79.536872     391.996283
8   Pipe_17  -405.530240   134.493077    -809.009471      -2.051009
9   Pipe_18  -777.124479   242.406106   -1504.342797     -49.906162
10  Pipe_19  -794.791479   247.168817   -1536.297930     -53.285029
11   Pipe_2  5386.296760  1649.485821     437.839298   10334.75422

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined from previous steps
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

# Compute global pipe std for minimum threshold
global_pipe_std = pipe_stats_df['actual_std'].mean()

# Apply a minimum std to avoid overly narrow ranges (10% of global pipe std)
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges using pipe-level stats with fixed k
k = 3  # Fixed number of standard deviations for plausible range
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
print("Per-pipe plausible ranges (fixed k):\n", pipe_stats_df[['pipe_id', 'actual_mean', 'actual_std', 'min_plausible', 'max_plausible']])

# Merge the plausible ranges into the main DataFrame
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(merged_df), 1):
    print(f"Processing Fold {fold}")
    train_df = merged_df.iloc[train_index].copy()
    test_df = merged_df.iloc[test_index].copy()
    print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

    features = ['measurement', 'diameter', 'length', 'roughness']
    X_cluster_train = train_df[features]
    scaler = RobustScaler()
    X_scaled_train = scaler.fit_transform(X_cluster_train)

    # Determine optimal number of clusters using the elbow method on training set
    wcss = []
    max_clusters = 11
    for i in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(X_scaled_train)
        wcss.append(kmeans.inertia_)

    # Plot the elbow curve for this fold (optional, can be disabled for efficiency)
    plt.figure(figsize=(10, 5))
    plt.plot(range(2, max_clusters + 1), wcss, marker='o')
    plt.title(f'Elbow Method for Optimal Number of Clusters - Fold {fold}')
    plt.xlabel('Number of Clusters')
    plt.ylabel('WCSS')
    plt.savefig(f'/content/drive/My Drive/elbow_curve_flow_fold_{fold}.png')
    plt.close()

    # Choose the optimal number of clusters (based on previous analysis)
    optimal_clusters = 7
    for i in range(2, len(wcss) - 1):
        if (wcss[i-1] - wcss[i]) / (wcss[i] - wcss[i+1]) < 0.5:
            optimal_clusters = i + 2
            break
    print(f"Optimal number of clusters for Fold {fold}: {optimal_clusters}")

    # Perform K-means clustering with optimal number of clusters on training set
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    train_cluster_labels = kmeans.fit_predict(X_scaled_train)

    # Assign clusters to test set using the trained K-means model
    X_cluster_test = test_df[features]
    X_scaled_test = scaler.transform(X_cluster_test)
    test_cluster_labels = kmeans.predict(X_scaled_test)

    # Add cluster labels to train and test dataframes
    train_df['cluster'] = train_cluster_labels
    test_df['cluster'] = test_cluster_labels

    # Combine train and test dataframes for this fold
    fold_df = pd.concat([train_df, test_df], axis=0)

    # Compute cluster statistics (mu, sigma, count, and associated pipes)
    cluster_stats = train_df.groupby('cluster').agg({
        'measurement': ['mean', 'std', 'count'],
        'pipe_id': lambda x: list(x.unique())
    }).reset_index()
    cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
    cluster_stats['cluster_mu'] = cluster_stats['cluster_mu'].round(2)
    cluster_stats['cluster_sigma'] = cluster_stats['cluster_sigma'].round(2)
    print("\nCluster Statistics (Training Set, Fold {fold}):")
    print(cluster_stats[['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']])

    cluster_means = train_df.groupby('cluster')['measurement'].mean().to_dict()
    cluster_stds = train_df.groupby('cluster')['measurement'].std().to_dict()
    fold_df['cluster_mu'] = fold_df['cluster'].map(cluster_means)
    fold_df['cluster_sigma'] = fold_df['cluster'].map(cluster_stds).fillna(global_pipe_std)

    fold_df['Z_score'] = (fold_df['measurement'] - fold_df['cluster_mu']) / fold_df['cluster_sigma'].replace(0, np.nan)
    fold_df['traditional_Z_score'] = (fold_df['measurement'] - global_flow_mean) / global_flow_std

    threshold = 3
    fold_df['outside_plausible'] = (fold_df['measurement'] < fold_df['min_plausible']) | (fold_df['measurement'] > fold_df['max_plausible'])
    fold_df['is_anomaly'] = np.abs(fold_df['Z_score']) > threshold
    fold_df['is_anomaly_traditional'] = np.abs(fold_df['traditional_Z_score']) > threshold
    fold_df['is_noise'] = fold_df['is_anomaly'] & fold_df['outside_plausible']
    fold_df['is_noise_traditional'] = fold_df['is_anomaly_traditional'] & fold_df['outside_plausible']

    numerical_cols = fold_df.select_dtypes(include=['float64', 'int64']).columns
    fold_df[numerical_cols] = fold_df[numerical_cols].round(2)

    def introduce_gaussian_noise(df, noise_level=1495.64, noise_fraction=0.05):
        noisy_df = df.copy()
        mask = np.random.rand(len(noisy_df)) < noise_fraction
        noise = np.random.normal(0, noise_level, size=len(noisy_df))
        noisy_df['measurement'] = noisy_df['measurement'] + noise * mask
        noisy_df['is_true_noise'] = mask.astype(int)
        return noisy_df

    noise_fractions = [0.02, 0.05, 0.10, 0.20]

    for noise_fraction in noise_fractions:
        print(f"Processing noise fraction: {noise_fraction*100}% in Fold {fold}")
        noisy_df = introduce_gaussian_noise(fold_df, noise_level=global_flow_std, noise_fraction=noise_fraction)

        noisy_df['Z_score'] = (noisy_df['measurement'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
        noisy_df['traditional_Z_score'] = (noisy_df['measurement'] - global_flow_mean) / global_flow_std
        noisy_df['outside_plausible'] = (noisy_df['measurement'] < noisy_df['min_plausible']) | (noisy_df['measurement'] > noisy_df['max_plausible'])
        noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
        noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']
        noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)
        y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid, zero_division=0)
        recall_hybrid = recall_score(y_true, y_pred_hybrid, zero_division=0)
        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)
        precision_traditional = precision_score(y_true, y_pred_traditional, zero_division=0)
        recall_traditional = recall_score(y_true, y_pred_traditional, zero_division=0)
        f1_traditional = f1_score(y_true, y_pred_traditional, zero_division=0)

        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        traditional_detected = y_pred_traditional[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
        traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        results_list.append({
            'Fold': fold,
            'Algorithm': 'K-means',
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Hybrid (Cluster-based)',
            'Precision': precision_hybrid,
            'Recall': recall_hybrid,
            'F1 Score': f1_hybrid,
            'Noise Detected (%)': hybrid_noise_percentage
        })
        results_list.append({
            'Fold': fold,
            'Algorithm': 'K-means',
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Traditional (Global)',
            'Precision': precision_traditional,
            'Recall': recall_traditional,
            'F1 Score': f1_traditional,
            'Noise Detected (%)': traditional_noise_percentage
        })

results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df.groupby(['Noise Fraction (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean',
    'Noise Detected (%)': 'mean'
}).reset_index())

try:
    noisy_df.to_csv('/content/drive/My Drive/noisy_results_kmeans_flow.csv', index=False)
    results_df.to_csv('/content/drive/My Drive/evaluation_results_kmeans_flow.csv', index=False)
    print("Results saved to '/content/drive/My Drive/'.")
except Exception as e:
    print(f"Error saving files: {e}")

print("\nZ-score stats (noisy dataset, last noise level of last fold):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced (last noise level of last fold): {total_noise_points}")
print(f"Hybrid method detected (last noise level of last fold): {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected (last noise level of last fold): {traditional_detected} ({traditional_noise_percentage:.2f}%)")

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Per-pipe plausible ranges (fixed k):
     pipe_id  actual_mean   actual_std  min_plausible  max_plausible
0    Pipe_1  5627.037288  1718.679788     470.997924   10783.076651
1   Pipe_10   550.314267   195.561336     -36.369742    1136.998276
2   Pipe_11   412.248822   162.662078     -75.737413     900.235057
3   Pipe_12   253.255767    83.374362       3.132682     503.378853
4   Pipe_13   266.276404    99.849458     -33.271971     565.824779
5   Pipe_14    88.363952    60.337333     -92.648046     269.375950
6   Pipe_15    13.391692    56.662783    -156.596656     183.380040
7   Pipe_16   156.229705    78.588859     -79.536872     391.996283
8   Pipe_17  -405.530240   134.493077    -809.009471      -2.051009
9   Pipe_18  -777.124479   242.406106   -1504.342797     -49.906162
10  Pipe_19  -794.791479   247.168817   -1536.297930     -53.285029
11   Pipe_2  5386.296760  1649.485821     437.839298   10334.75422

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
from scipy.stats import gennorm

warnings.filterwarnings('ignore')

# === Data Loading and Merging ===
# (Assumes combined_df and pipe_df are defined elsewhere in your workflow)
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global statistics
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Per-pipe statistics and plausible ranges
tube_stats = merged_df.groupby('pipe_id')['measurement'].agg(['mean','std','count']).reset_index()
tube_stats.columns = ['pipe_id','actual_mean','actual_std','sample_count']
tube_stats = tube_stats[tube_stats['sample_count'] >= 100]

global_pipe_std = tube_stats['actual_std'].mean()
tube_stats['actual_std'] = tube_stats['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))
k = 3
tube_stats['min_plausible'] = tube_stats['actual_mean'] - k * tube_stats['actual_std']
tube_stats['max_plausible'] = tube_stats['actual_mean'] + k * tube_stats['actual_std']

merged_df = merged_df.merge(
    tube_stats[['pipe_id','min_plausible','max_plausible']],
    on='pipe_id', how='left'
)

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(merged_df), 1):
    print(f"Processing Fold {fold}")
    train_df = merged_df.iloc[train_index].copy()
    test_df = merged_df.iloc[test_index].copy()
    print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

    # Features and scaling
    features = ['measurement','diameter','length','roughness']
    scaler = RobustScaler()
    X_train = scaler.fit_transform(train_df[features])

    # Elbow plot for this fold
    wcss = []
    for i in range(2, 12):
        km = KMeans(n_clusters=i, random_state=42)
        km.fit(X_train)
        wcss.append(km.inertia_)

    plt.figure(figsize=(8, 4))
    plt.plot(range(2, 12), wcss, 'o-')
    plt.xlabel('Clusters'); plt.ylabel('WCSS'); plt.title(f'Elbow Method - Fold {fold}')
    plt.savefig(f'elbow_kmeans_fold_{fold}.png'); plt.close()

    # Fit KMeans with chosen clusters
    optimal_clusters = 7
    for i in range(2, len(wcss) - 1):
        if (wcss[i-1] - wcss[i]) / (wcss[i] - wcss[i+1]) < 0.5:
            optimal_clusters = i + 2
            break
    print(f"Optimal number of clusters for Fold {fold}: {optimal_clusters}")

    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    train_df['cluster'] = kmeans.fit_predict(scaler.transform(train_df[features]))
    test_df['cluster'] = kmeans.predict(scaler.transform(test_df[features]))
    fold_df = pd.concat([train_df, test_df], ignore_index=True)

    # Cluster statistics for Z-score
    stats = train_df.groupby('cluster')['measurement'].agg(['mean','std']).rename(columns={'mean':'mu','std':'sigma'})
    fold_df['cluster_mu'] = fold_df['cluster'].map(stats['mu'])
    fold_df['cluster_sigma'] = fold_df['cluster'].map(stats['sigma']).fillna(global_pipe_std)

    # Z-scores and traditional scores
    fold_df['Z_score'] = (fold_df['measurement'] - fold_df['cluster_mu']) / fold_df['cluster_sigma']
    fold_df['traditional_Z_score'] = (fold_df['measurement'] - global_flow_mean) / global_flow_std

    # Plausibility and anomaly labels
    threshold = 3
    fold_df['outside_plausible'] = (
        (fold_df['measurement'] < fold_df['min_plausible']) |
        (fold_df['measurement'] > fold_df['max_plausible'])
    )
    fold_df['is_anomaly'] = fold_df['Z_score'].abs() > threshold
    fold_df['is_anomaly_traditional'] = fold_df['traditional_Z_score'].abs() > threshold
    fold_df['is_noise'] = fold_df['is_anomaly'] & fold_df['outside_plausible']
    fold_df['is_noise_traditional'] = fold_df['is_anomaly_traditional'] & fold_df['outside_plausible']

    # Function to introduce Generalized Normal Noise (GNN)
    def introduce_gnn_noise(df, beta=1.5, scale=global_flow_std, noise_fraction=0.05):
        noisy = df.copy()
        mask = np.random.rand(len(noisy)) < noise_fraction
        noise = gennorm.rvs(beta, loc=0, scale=scale, size=len(noisy))
        noisy['measurement'] += noise * mask
        noisy['is_true_noise'] = mask.astype(int)
        return noisy

    # Evaluate across noise fractions
    noise_fractions = [0.02, 0.05, 0.1, 0.2]
    for frac in noise_fractions:
        print(f"GNN noise fraction: {frac*100}% in Fold {fold}")
        df_noisy = introduce_gnn_noise(fold_df, beta=1.5, scale=global_flow_std, noise_fraction=frac)
        df_noisy['Z_score'] = (df_noisy['measurement'] - df_noisy['cluster_mu']) / df_noisy['cluster_sigma']
        df_noisy['is_anomaly'] = df_noisy['Z_score'].abs() > threshold
        df_noisy['outside_plausible'] = (
            (df_noisy['measurement'] < df_noisy['min_plausible']) |
            (df_noisy['measurement'] > df_noisy['max_plausible'])
        )
        df_noisy['is_noise'] = df_noisy['is_anomaly'] & df_noisy['outside_plausible']

        df_noisy['traditional_Z_score'] = (df_noisy['measurement'] - global_flow_mean) / global_flow_std
        df_noisy['is_anomaly_traditional'] = df_noisy['traditional_Z_score'].abs() > threshold
        df_noisy['is_noise_traditional'] = df_noisy['is_anomaly_traditional'] & df_noisy['outside_plausible']

        y_true = df_noisy['is_true_noise']
        for method, label in [('Hybrid', 'is_noise'), ('Traditional', 'is_noise_traditional')]:
            y_pred = df_noisy[label].astype(int)
            prec = precision_score(y_true, y_pred, zero_division=0)
            rec = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)
            detected = y_pred[y_true==1].sum()
            total = y_true.sum()
            perc = (detected/total*100) if total>0 else 0
            results.append({
                'Fold': fold,
                'Noise (%)': frac*100,
                'Method': method,
                'Precision': prec,
                'Recall': rec,
                'F1': f1,
                'Detected (%)': perc
            })

results_df = pd.DataFrame(results)
print("\nResults:\n", results_df.groupby(['Noise (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1': 'mean',
    'Detected (%)': 'mean'
}).reset_index())

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Processing Fold 1
Training set size: 4765440, Test set size: 1191360
Optimal number of clusters for Fold 1: 7
GNN noise fraction: 2.0% in Fold 1
GNN noise fraction: 5.0% in Fold 1
GNN noise fraction: 10.0% in Fold 1
GNN noise fraction: 20.0% in Fold 1
Processing Fold 2
Training set size: 4765440, Test set size: 1191360
Optimal number of clusters for Fold 2: 7
GNN noise fraction: 2.0% in Fold 2
GNN noise fraction: 5.0% in Fold 2
GNN noise fraction: 10.0% in Fold 2
GNN noise fraction: 20.0% in Fold 2
Processing Fold 3
Training set size: 4765440, Test set size: 1191360
Optimal number of clusters for Fold 3: 7
GNN noise fraction: 2.0% in Fold 3
GNN noise fraction: 5.0% in Fold 3
GNN noise fraction: 10.0% in Fold 3
GNN noise fraction: 20.0% in Fold 3
Processing Fold 4
Training set size: 4765440, Test set size: 1191360
Optimal number of clusters for Fold 4: 7
GNN noise fraction: 2.0% in Fold 4
GNN noise fraction:

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined from previous steps
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

# Compute global pipe std for minimum threshold
global_pipe_std = pipe_stats_df['actual_std'].mean()

# Apply a minimum std to avoid overly narrow ranges (10% of global pipe std)
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges using pipe-level stats with fixed k
k = 3  # Fixed number of standard deviations for plausible range
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
print("Per-pipe plausible ranges (fixed k):\n", pipe_stats_df[['pipe_id', 'actual_mean', 'actual_std', 'min_plausible', 'max_plausible']])

# Merge the plausible ranges into the main DataFrame
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Subsample the data to reduce memory usage (scale up to 50% after tuning)
subsample_fraction = 0.2  # Use 20% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(subsampled_df), 1):
    print(f"Processing Fold {fold}")
    train_df = subsampled_df.iloc[train_index].copy()
    test_df = subsampled_df.iloc[test_index].copy()
    print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

    # Prepare data for clustering on training set
    features = ['measurement', 'diameter', 'length', 'roughness']
    X_cluster_train = train_df[features]
    scaler = RobustScaler()
    X_scaled_train = scaler.fit_transform(X_cluster_train)

    # Prepare data for test set
    X_cluster_test = test_df[features]
    X_scaled_test = scaler.transform(X_cluster_test)

    # Generate k-distance plot to determine initial eps
    min_samples = 50  # Default min_samples for k-distance plot
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
    distances, indices = neighbors.kneighbors(X_scaled_train)
    k_distances = np.sort(distances[:, min_samples - 1])
    plt.figure(figsize=(10, 6))
    plt.plot(k_distances)
    plt.title(f'K-Distance Plot (Distance to {min_samples}th Nearest Neighbor) - Fold {fold}')
    plt.xlabel('Points Sorted by Distance')
    plt.ylabel('Distance')
    plt.grid(True)
    plt.savefig(f'/content/drive/My Drive/k_distance_plot_flow_fold_{fold}.png')
    plt.close()
    print(f"K-distance plot saved to '/content/drive/My Drive/k_distance_plot_flow_fold_{fold}.png'.")

    # Suggest eps based on the 95th percentile of k-distances
    suggested_eps = np.percentile(k_distances, 95)
    print(f"Suggested eps based on 95th percentile of k-distances for Fold {fold}: {suggested_eps:.3f}")

    # Parameter tuning using silhouette score
    eps_values = np.arange(0.005, 0.015, 0.002)  # Adjusted range based on suggested eps
    min_samples_values = [20, 30, 50, 60, 70]
    best_score = -1
    best_params = {'eps': 0, 'min_samples': 0}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(X_scaled_train)
            if len(np.unique(labels)) > 1 and -1 in labels:  # Ensure multiple clusters and noise points
                score = silhouette_score(X_scaled_train, labels, sample_size=5000)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}

    print(f"Best parameters for Fold {fold}: eps={best_params['eps']:.3f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")

    # Use best parameters for clustering
    dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    train_cluster_labels = dbscan.fit_predict(X_scaled_train)
    test_cluster_labels = dbscan.fit_predict(X_scaled_test)

    # Add cluster labels to train and test dataframes
    train_df['cluster'] = train_cluster_labels
    test_df['cluster'] = test_cluster_labels
    fold_df = pd.concat([train_df, test_df], axis=0)

    # Compute cluster statistics (mu, sigma, count, and associated pipes) for non-noise clusters
    cluster_stats = train_df[train_df['cluster'] != -1].groupby('cluster').agg({
        'measurement': ['mean', 'std', 'count'],
        'pipe_id': lambda x: list(x.unique())  # Get unique pipes in each cluster
    }).reset_index()
    cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
    cluster_stats['cluster_mu'] = cluster_stats['cluster_mu'].round(2)
    cluster_stats['cluster_sigma'] = cluster_stats['cluster_sigma'].round(2)

    # Merge small clusters (point_count < 100) into noise (-1)
    min_cluster_size = 100
    small_clusters = cluster_stats[cluster_stats['point_count'] < min_cluster_size]['cluster'].tolist()
    fold_df.loc[fold_df['cluster'].isin(small_clusters), 'cluster'] = -1
    train_df.loc[train_df['cluster'].isin(small_clusters), 'cluster'] = -1
    test_df.loc[test_df['cluster'].isin(small_clusters), 'cluster'] = -1

    # Recompute cluster statistics after merging small clusters
    cluster_stats = train_df[train_df['cluster'] != -1].groupby('cluster').agg({
        'measurement': ['mean', 'std', 'count'],
        'pipe_id': lambda x: list(x.unique())
    }).reset_index()
    cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
    cluster_stats['cluster_mu'] = cluster_stats['cluster_mu'].round(2)
    cluster_stats['cluster_sigma'] = cluster_stats['cluster_sigma'].round(2)

    # Report noise points
    noise_count = len(train_df[train_df['cluster'] == -1])
    print(f"\nNumber of noise points (cluster -1) in training set (Fold {fold}): {noise_count} ({noise_count/len(train_df)*100:.2f}%)")

    print(f"\nCluster Statistics (Training Set, Non-Noise Clusters, Fold {fold}):")
    print(cluster_stats[['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']])

    # Assign cluster mean and std for Z-score calculation
    cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['measurement'].mean().to_dict()
    cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['measurement'].std().to_dict()
    # Apply minimum cluster_sigma to avoid extreme Z-scores
    min_sigma = global_pipe_std * 0.1  # 10% of global pipe std
    cluster_stds = {k: max(v, min_sigma) for k, v in cluster_stds.items()}
    fold_df['cluster_mu'] = fold_df['cluster'].map(cluster_means)
    fold_df['cluster_sigma'] = fold_df['cluster'].map(cluster_stds).fillna(global_pipe_std)
    # For noise points (cluster -1), use global statistics
    fold_df.loc[fold_df['cluster'] == -1, 'cluster_mu'] = global_flow_mean
    fold_df.loc[fold_df['cluster'] == -1, 'cluster_sigma'] = global_flow_std

    fold_df['Z_score'] = (fold_df['measurement'] - fold_df['cluster_mu']) / fold_df['cluster_sigma'].replace(0, np.nan)
    fold_df['traditional_Z_score'] = (fold_df['measurement'] - global_flow_mean) / global_flow_std

    threshold = 3
    fold_df['outside_plausible'] = (fold_df['measurement'] < fold_df['min_plausible']) | (fold_df['measurement'] > fold_df['max_plausible'])
    fold_df['is_anomaly'] = np.abs(fold_df['Z_score']) > threshold
    fold_df['is_anomaly_traditional'] = np.abs(fold_df['traditional_Z_score']) > threshold
    fold_df['is_noise'] = fold_df['is_anomaly'] & fold_df['outside_plausible']
    fold_df['is_noise_traditional'] = fold_df['is_anomaly_traditional'] & fold_df['outside_plausible']

    numerical_cols = fold_df.select_dtypes(include=['float64', 'int64']).columns
    fold_df[numerical_cols] = fold_df[numerical_cols].round(2)

    def introduce_gaussian_noise(df, noise_level=global_flow_std, noise_fraction=0.05):
        noisy_df = df.copy()
        mask = np.random.rand(len(noisy_df)) < noise_fraction
        noise = np.random.normal(0, noise_level, size=len(noisy_df))
        noisy_df['measurement'] = noisy_df['measurement'] + noise * mask
        noisy_df['is_true_noise'] = mask.astype(int)
        return noisy_df

    # Evaluate across noise fractions
    noise_fractions = [0.02, 0.05, 0.10, 0.20]
    for noise_fraction in noise_fractions:
        print(f"Processing noise fraction: {noise_fraction*100}% in Fold {fold}")
        noisy_df = introduce_gaussian_noise(fold_df, noise_level=global_flow_std, noise_fraction=noise_fraction)

        noisy_df['Z_score'] = (noisy_df['measurement'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
        noisy_df['traditional_Z_score'] = (noisy_df['measurement'] - global_flow_mean) / global_flow_std
        noisy_df['outside_plausible'] = (noisy_df['measurement'] < noisy_df['min_plausible']) | (noisy_df['measurement'] > noisy_df['max_plausible'])
        noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
        noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']
        noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)
        y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid, zero_division=0)
        recall_hybrid = recall_score(y_true, y_pred_hybrid, zero_division=0)
        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)
        precision_traditional = precision_score(y_true, y_pred_traditional, zero_division=0)
        recall_traditional = recall_score(y_true, y_pred_traditional, zero_division=0)
        f1_traditional = f1_score(y_true, y_pred_traditional, zero_division=0)

        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        traditional_detected = y_pred_traditional[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
        traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Hybrid (Cluster-based)',
            'Precision': precision_hybrid,
            'Recall': recall_hybrid,
            'F1 Score': f1_hybrid,
            'Noise Detected (%)': hybrid_noise_percentage
        })
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Traditional (Global)',
            'Precision': precision_traditional,
            'Recall': recall_traditional,
            'F1 Score': f1_traditional,
            'Noise Detected (%)': traditional_noise_percentage
        })

results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df.groupby(['Noise Fraction (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean',
    'Noise Detected (%)': 'mean'
}).reset_index())

try:
    noisy_df.to_csv(f'/content/drive/My Drive/noisy_results_dbscan_flow_fold_{fold}.csv', index=False)
    results_df.to_csv('/content/drive/My Drive/evaluation_results_dbscan_flow.csv', index=False)
    print("Results saved to '/content/drive/My Drive/'.")
except Exception as e:
    print(f"Error saving files: {e}")

print("\nZ-score stats (noisy dataset, last noise level of last fold):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced (last noise level of last fold): {total_noise_points}")
print(f"Hybrid method detected (last noise level of last fold): {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected (last noise level of last fold): {traditional_detected} ({traditional_noise_percentage:.2f}%)")

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Per-pipe plausible ranges (fixed k):
     pipe_id  actual_mean   actual_std  min_plausible  max_plausible
0    Pipe_1  5627.037288  1718.679788     470.997924   10783.076651
1   Pipe_10   550.314267   195.561336     -36.369742    1136.998276
2   Pipe_11   412.248822   162.662078     -75.737413     900.235057
3   Pipe_12   253.255767    83.374362       3.132682     503.378853
4   Pipe_13   266.276404    99.849458     -33.271971     565.824779
5   Pipe_14    88.363952    60.337333     -92.648046     269.375950
6   Pipe_15    13.391692    56.662783    -156.596656     183.380040
7   Pipe_16   156.229705    78.588859     -79.536872     391.996283
8   Pipe_17  -405.530240   134.493077    -809.009471      -2.051009
9   Pipe_18  -777.124479   242.406106   -1504.342797     -49.906162
10  Pipe_19  -794.791479   247.168817   -1536.297930     -53.285029
11   Pipe_2  5386.296760  1649.485821     437.839298   10334.75422

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
from sklearn.neighbors import NearestNeighbors
import warnings
from scipy.stats import gennorm

warnings.filterwarnings('ignore')

# Assuming combined_df and pipe_df are already defined
merged_df = pd.merge(combined_df, pipe_df, on='pipe_id', how='inner')
print(f"Merged data size: {len(merged_df)} rows.")

# Compute global flow stats
global_flow_mean = merged_df['measurement'].mean()
global_flow_std = merged_df['measurement'].std()
print(f"Global flow stats: Mean = {global_flow_mean:.2f}, Std = {global_flow_std:.2f}")

# Compute per-pipe statistics
pipe_stats_df = merged_df.groupby('pipe_id')['measurement'].agg(['mean', 'std', 'count']).reset_index()
pipe_stats_df.columns = ['pipe_id', 'actual_mean', 'actual_std', 'sample_count']
pipe_stats_df = pipe_stats_df[pipe_stats_df['sample_count'] >= 100]

global_pipe_std = pipe_stats_df['actual_std'].mean()
pipe_stats_df['actual_std'] = pipe_stats_df['actual_std'].apply(lambda x: max(x, global_pipe_std * 0.1))

# Compute plausible ranges
k = 3
pipe_stats_df['min_plausible'] = pipe_stats_df['actual_mean'] - k * pipe_stats_df['actual_std']
pipe_stats_df['max_plausible'] = pipe_stats_df['actual_mean'] + k * pipe_stats_df['actual_std']
merged_df = merged_df.merge(pipe_stats_df[['pipe_id', 'min_plausible', 'max_plausible']], on='pipe_id', how='left')

# Subsample
subsample_fraction = 0.2
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(subsampled_df), 1):
    print(f"Processing Fold {fold}")
    train_df = subsampled_df.iloc[train_index].copy()
    test_df = subsampled_df.iloc[test_index].copy()

    # Scale features
    features = ['measurement', 'diameter', 'length', 'roughness']
    scaler = RobustScaler()
    X_train = scaler.fit_transform(train_df[features])
    X_test = scaler.transform(test_df[features])

    # K-distance plot
    min_samples = 50
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_train)
    dists, _ = neighbors.kneighbors(X_train)
    k_distances = np.sort(dists[:, min_samples - 1])
    plt.figure(figsize=(10, 6))
    plt.plot(k_distances)
    plt.title(f'K-Distance Plot (Distance to {min_samples}th NN) - Fold {fold}')
    plt.xlabel('Points Sorted')
    plt.ylabel('Distance')
    plt.savefig(f'k_distance_plot_fold_{fold}.png')
    plt.close()

    # Parameter tuning
    suggested_eps = np.percentile(k_distances, 95)
    eps_values = np.arange(suggested_eps * 0.5, suggested_eps * 1.5, suggested_eps * 0.1)
    min_samples_values = [20, 30, 50]
    best_score = -1
    best_params = {'eps': None, 'min_samples': None}
    for eps in eps_values:
        for ms in min_samples_values:
            labels = DBSCAN(eps=eps, min_samples=ms).fit_predict(X_train)
            if len(set(labels)) > 1 and -1 in labels:
                score = silhouette_score(X_train, labels, sample_size=2000)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': ms}
    print(f"Best DBSCAN params for Fold {fold}: {best_params}, silhouette={best_score:.3f}")

    # Final clustering
    db = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    train_df['cluster'] = db.fit_predict(X_train)
    test_df['cluster'] = db.fit_predict(X_test)

    # Combine data for this fold
    merged_all = pd.concat([train_df, test_df], axis=0)

    # Compute cluster stats
    cluster_stats = train_df[train_df['cluster'] != -1].groupby('cluster').agg({
        'measurement': ['mean', 'std', 'count'],
        'pipe_id': lambda x: list(x.unique())
    }).reset_index()
    cluster_stats.columns = ['cluster', 'cluster_mu', 'cluster_sigma', 'point_count', 'pipes']
    # Merge small clusters into noise
    min_cluster_size = 100
    small_clusters = cluster_stats[cluster_stats['point_count'] < min_cluster_size]['cluster']
    merged_all.loc[merged_all['cluster'].isin(small_clusters), 'cluster'] = -1

    # Map cluster stats
    dict_mu = cluster_stats.set_index('cluster')['cluster_mu'].to_dict()
    dict_sigma = cluster_stats.set_index('cluster')['cluster_sigma'].to_dict()
    merged_all['cluster_mu'] = merged_all['cluster'].map(dict_mu).fillna(global_flow_mean)
    merged_all['cluster_sigma'] = merged_all['cluster'].map(dict_sigma).fillna(global_flow_std)

    # Z-scores and flags
    threshold = 3
    merged_all['Z_score'] = (merged_all['measurement'] - merged_all['cluster_mu']) / merged_all['cluster_sigma']
    merged_all['Z_trad'] = (merged_all['measurement'] - global_flow_mean) / global_flow_std
    merged_all['outside_plausible'] = (
        merged_all['measurement'] < merged_all['min_plausible']
    ) | (
        merged_all['measurement'] > merged_all['max_plausible']
    )
    merged_all['is_anomaly'] = merged_all['Z_score'].abs() > threshold
    merged_all['is_anomaly_traditional'] = merged_all['Z_trad'].abs() > threshold
    merged_all['is_noise'] = merged_all['is_anomaly'] & merged_all['outside_plausible']
    merged_all['is_noise_traditional'] = merged_all['is_anomaly_traditional'] & merged_all['outside_plausible']

    # --- Noise Introduction using GNN ---
    def introduce_gnn_noise(df, beta=1.5, scale=global_flow_std, noise_fraction=0.05):
        noisy_df = df.copy()
        mask = np.random.rand(len(noisy_df)) < noise_fraction
        noise = gennorm.rvs(beta, loc=0, scale=scale, size=len(noisy_df))
        noisy_df['measurement'] += noise * mask
        noisy_df['is_true_noise'] = mask.astype(int)
        return noisy_df

    # Evaluation loop
    noise_fractions = [0.02, 0.05, 0.10, 0.20]
    for noise_fraction in noise_fractions:
        print(f"Processing GNN noise fraction: {noise_fraction*100}% in Fold {fold}")
        noisy_df = introduce_gnn_noise(
            merged_all,
            beta=1.5,
            scale=global_flow_std,
            noise_fraction=noise_fraction
        )

        # Recompute detection flags
        noisy_df['Z_score'] = (noisy_df['measurement'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma']
        noisy_df['Z_trad'] = (noisy_df['measurement'] - global_flow_mean) / global_flow_std
        noisy_df['outside_plausible'] = (
            noisy_df['measurement'] < noisy_df['min_plausible']
        ) | (
            noisy_df['measurement'] > noisy_df['max_plausible']
        )
        noisy_df['is_anomaly'] = noisy_df['Z_score'].abs() > threshold
        noisy_df['is_anomaly_traditional'] = noisy_df['Z_trad'].abs() > threshold
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']
        noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

        # Metrics
        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)
        y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid, zero_division=0)
        recall_hybrid = recall_score(y_true, y_pred_hybrid, zero_division=0)
        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)
        precision_traditional = precision_score(y_true, y_pred_traditional, zero_division=0)
        recall_traditional = recall_score(y_true, y_pred_traditional, zero_division=0)
        f1_traditional = f1_score(y_true, y_pred_traditional, zero_division=0)

        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        traditional_detected = y_pred_traditional[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
        traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Hybrid (Cluster-based)',
            'Precision': precision_hybrid,
            'Recall': recall_hybrid,
            'F1 Score': f1_hybrid,
            'Noise Detected (%)': hybrid_noise_percentage
        })
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Traditional (Global)',
            'Precision': precision_traditional,
            'Recall': recall_traditional,
            'F1 Score': f1_traditional,
            'Noise Detected (%)': traditional_noise_percentage
        })

# Consolidate and save
results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df.groupby(['Noise Fraction (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean',
    'Noise Detected (%)': 'mean'
}).reset_index())

Merged data size: 5956800 rows.
Global flow stats: Mean = 829.88, Std = 1495.64
Processing Fold 1
Best DBSCAN params for Fold 1: {'eps': np.float64(0.003174603174604051), 'min_samples': 20}, silhouette=0.582
Processing GNN noise fraction: 2.0% in Fold 1
Processing GNN noise fraction: 5.0% in Fold 1
Processing GNN noise fraction: 10.0% in Fold 1
Processing GNN noise fraction: 20.0% in Fold 1
Processing Fold 2
Best DBSCAN params for Fold 2: {'eps': np.float64(0.004126984126985266), 'min_samples': 20}, silhouette=0.598
Processing GNN noise fraction: 2.0% in Fold 2
Processing GNN noise fraction: 5.0% in Fold 2
Processing GNN noise fraction: 10.0% in Fold 2
Processing GNN noise fraction: 20.0% in Fold 2
Processing Fold 3
Best DBSCAN params for Fold 3: {'eps': np.float64(0.004444444444445672), 'min_samples': 20}, silhouette=0.585
Processing GNN noise fraction: 2.0% in Fold 3
Processing GNN noise fraction: 5.0% in Fold 3
Processing GNN noise fraction: 10.0% in Fold 3
Processing GNN noise frac